### Imports

In [179]:
import os
import pandas as pd
from typing import List
from pypdf import PdfReader

# Langchain,
from langchain_community.chat_models import ChatOllama
from langchain_community.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


### Extracting from PDF

In [180]:
# Opening PDF,
reader = PdfReader("statement.pdf")
n_pages = len(reader.pages)

# Extracting all text from the PDF,
pdf_text = ""
for i in range(n_pages):
    page = reader.pages[i]
    page_text = page.extract_text()
    pdf_text += page_text

In [209]:
import pymupdf # imports the pymupdf library

pdf_text = ""
doc = pymupdf.open("statement.pdf") # open a document
for page in doc: # iterate the document pages
  text = page.get_text() # get plain text encoded as UTF-8
  pdf_text += text

### Model Wrapper

In [ ]:
# Wrapping around LLM,
TEMPERATURE = 0.05
KEY = ""
os.environ["OPENAI_API_KEY"] = KEY
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=TEMPERATURE)

### Prompt Engineering

In [213]:
# Creating schema for LLM output,
class Transaction(BaseModel):
    date: str = Field(description="The date the transaction occured.")
    description: str = Field(description="The description of the transcation. Includes the person/organisation involves, the reference and sometimes the location. Do NOT include the transaction type.")
    type: str = Field(description="The type of transaction. Sometimes given as a code.")
    paid_in: float = Field(description="Money paid IN the account.")
    paid_out: float = Field(description="Money paid OUT of the account.")

class Summary(BaseModel):
    opening_balance: float
    payments_in: float
    payments_out: float
    closing_balance: float

class OutputWrapper(BaseModel):
    summary: Summary
    transactions: List[Transaction]

# Creating parser,
parser = PydanticOutputParser(pydantic_object=OutputWrapper)

# Generating formatting instructions from schemas,
format_instructions = parser.get_format_instructions()

# Creating the LLM prompt,
PROMPT = f"""You have been given raw text that has been extracted from the PDF of a bank statement. Extract the balance summary and transactions 
             details structured in JSON with the following schema: {format_instructions}. The statement text is: {pdf_text}. RESPOND ONLY IN JSON FORMAT"""

PROMPT

'You have been given raw text that has been extracted from the PDF of a bank statement. Extract the balance summary and transactions \n             details structured in JSON with the following schema: The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Summary": {"properties": {"opening_balance": {"title": "Opening Balance", "type": "number"}, "payments_in": {"title": "Payments In", "type": "number"}, "payments_out": {"title": "Payments Out", "type": "number"}, "closing_balance": {"title": "Closing Balance", "type": "number"}}, "required": ["opening_balance", "payments_in", 

### Chain

In [212]:
# Sending prompt to LLM,
response = llm.invoke([HumanMessage(content=PROMPT)])

# Parsing LLM output,
parsed_response = parser.parse(response.content)

# Creating dataframe and converting to CSV,
df = pd.DataFrame([transcation.model_dump() for transcation in parsed_response.transactions])

# Calculating closing balance after each transaction,
opening_balance = dict(parsed_response.summary)["opening_balance"]
closing_balance_vals = [opening_balance]

for index, row in df.iterrows():
    closing_balance_val = round(closing_balance_vals[-1] + row["paid_in"] - row["paid_out"], 2)
    closing_balance_vals.append(closing_balance_val)
closing_balance_vals.pop(0)

df["closing_balance"] = closing_balance_vals

# Printing,
print(dict(parsed_response.summary))
df

{'opening_balance': 67.66, 'payments_in': 428.36, 'payments_out': 376.28, 'closing_balance': 119.74}


,date,description,type,paid_in,paid_out,closing_balance
0,01/01/2024,DEPOSIT INTEREST Starling Bank (December Inter...,DEPOSIT INTEREST,0.36,0.00,68.02
1,01/01/2024,FASTER PAYMENT RAMRUCHA P (mum - new yrs gift),FASTER PAYMENT,50.00,0.00,118.02
2,02/01/2024,FASTER PAYMENT RAMRUCHA (dad),FASTER PAYMENT,0.00,13.00,105.02
3,03/01/2024,ONLINE PAYMENT Amazon Marketplace,ONLINE PAYMENT,0.00,12.99,92.03
4,04/01/2024,ATM Cash Machine,ATM,0.00,10.00,82.03
5,04/01/2024,CONTACTLESS Co-op,CONTACTLESS,0.00,8.20,73.83
6,04/01/2024,CONTACTLESS Co-op,CONTACTLESS,0.00,7.20,66.63
7,06/01/2024,CONTACTLESS Tesco,CONTACTLESS,0.00,38.32,28.31
8,07/01/2024,CONTACTLESS Co-op,CONTACTLESS,0.00,3.25,25.06
9,10/01/2024,CONTACTLESS Co-op,CONTACTLESS,0.00,9.05,16.01
